# Exploratory Data Analysis


- [ ]  Basic composition
    - [ ]  df.info
    - [ ]  change datatype
    - [ ]  bar graph: app vs review counts
    - [ ]  pie chart: attraction vs review count
    - [ ]  bar graph: year vs
    - [ ]  rating: side-by-side box plot, box plot for all

In [1]:
# Import Python Libraries
#Data Analysis 
import numpy as np
import pandas as pd
from datetime import datetime

#Visualization
import matplotlib.pyplot as plt
import matplotlib.style as style
from matplotlib.colors import ListedColormap
from matplotlib import cm
import seaborn as sns
%matplotlib inline

#Plotly Libraris
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
from dash_core_components import Dropdown
from IPython.display import display, HTML

#Warnings
import warnings
warnings.filterwarnings('ignore')

/var/folders/y2/gl6v6t_n63g4cqcss8z26cj00000gn/T/ipykernel_8986/4091279474.py:23: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`



## 1. Understanding Data

In [2]:
review_all = pd.read_csv('../data/all_reviews.csv')
review_all.head(5)

,platform,attraction,review,rating,datecolumn,coordinates
0,TripAdvisor,Alcatraz Island,My family took the tour ( BUY TICKETS IN ADVAN...,5.0,"Mar, 2023",NaN
1,TripAdvisor,Alcatraz Island,This is a must stop if you are in San Fran!!! ...,5.0,"Feb, 2023",NaN
2,TripAdvisor,Alcatraz Island,I did not expect to enjoy the tour as much as ...,5.0,"Mar, 2023",NaN
3,TripAdvisor,Alcatraz Island,San Francisco is completely unsafe. We bought ...,1.0,"Mar, 2023",NaN
4,TripAdvisor,Alcatraz Island,I had a 13-hour layover in San Francisco And I...,4.0,"Mar, 2023",NaN


This dataset...
- platform - 
- attraction -
- review -
- rating - 
- datecolumn - 
- coordinate - 


### 1.1 Overall Statistics

In [3]:
review_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10011 entries, 0 to 10010
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   platform     10011 non-null  object 
 1   attraction   10011 non-null  object 
 2   review       9927 non-null   object 
 3   rating       9365 non-null   float64
 4   datecolumn   6947 non-null   object 
 5   coordinates  150 non-null    object 
dtypes: float64(1), object(5)
memory usage: 469.4+ KB


Change data types for datecolumn, Remove null values based on reviews

In [4]:
# Convert the 'date' column to datetime format
review_all['datecolumn'] = pd.to_datetime(review_all['datecolumn'], format='%b, %Y')

# Drop rows with null values in the 'review' column
review_all = review_all.dropna(subset=['review'])

# Drop duplicate
review_all = review_all.drop_duplicates()
review_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9872 entries, 0 to 10010
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   platform     9872 non-null   object        
 1   attraction   9872 non-null   object        
 2   review       9872 non-null   object        
 3   rating       9228 non-null   float64       
 4   datecolumn   6940 non-null   datetime64[ns]
 5   coordinates  150 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 539.9+ KB


In [5]:
platforms = review_all['platform'].unique()
places = review_all['attraction'].unique()

In [22]:
fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number",
    value = len(platforms),
    title = {'text': "Platforms",'font': {'color': 'black','size':20}},
    number={'font':{'color': 'black','size':50}},
    domain = {'row': 0, 'column': 0}
))
fig.add_trace(go.Indicator(
    mode = "number",
    value = len(places),
    title = {'text': "Attractions",'font': {'color': 'brown','size':20}},
    number={'font':{'color': 'brown','size':50}},
    domain = {'row': 0, 'column': 1}
))
fig.add_trace(go.Indicator(
    mode = "number",
    value = len(review_all['review']),
    title = {'text': "Reviews",'font': {'color': 'green','size':20}},
    number={'font':{'color': 'green','size':50}},
    domain = {'row': 0, 'column': 2}
))
fig.update_layout(
    grid = {'rows': 1, 'columns': 3, 'pattern': "independent"})
fig.show()

### 1.2 Platform Wise Analysis

Distribution for platform, attraction

In [7]:
fig1 = px.pie(review_all, names='platform')
fig1.update_layout(title='Pie Charts of Platform')
fig1.show()

In [8]:
def platform_piechart(platform, column):
    t_place_count = review_all.loc[review_all['platform']==platform][column].value_counts().sort_values()
    fig = px.pie(t_place_count, 
             values=column, 
             names=t_place_count.index,
             title = f'{column}s from {platform} ')
    fig.show()

In [9]:
for platform in platforms:
    platform_piechart(platform,'attraction')

### 2.3 Attraction Wise Analysis

In [10]:
attraction_count = review_all['attraction'].value_counts()
fig = px.bar(attraction_count.head(10))
fig.update_xaxes(title='Place Name')
fig.update_yaxes(title='# of Reviews')
fig.update_layout(title='# of Reviews for Places (Top 10)')
fig.show()

### 2.4 Date Wise Analysis

In [11]:
review_all['month'] = review_all['datecolumn'].dt.month
review_all['year'] = review_all['datecolumn'].dt.year


In [12]:
year_count = review_all['year'].value_counts()
fig = px.bar(year_count)
fig.update_xaxes(title='Year')
fig.update_yaxes(title='# of Reviews')
fig.update_layout(title='# of Reviews by Year')
fig.show()

In [13]:
year2022 = review_all.loc[(review_all['datecolumn']>'2022-2-28') & (review_all['datecolumn']<'2023-3-1')]
month2022_count = year2022['month'].value_counts()
fig = px.bar(month2022_count)
fig.update_xaxes(title='Month')
fig.update_yaxes(title='# of Reviews')
fig.update_layout(title='# of Reviews by Month from Mar 2022 to Feb 2023')
fig.show()

### 2.5 Rating Wise Analysis

In [14]:
review_all.describe()['rating']

count    9228.000000
mean        4.426094
std         0.957343
min         1.000000
25%         4.000000
50%         5.000000
75%         5.000000
max         5.000000
Name: rating, dtype: float64

In [15]:
fig = px.box(review_all, y='rating')
fig.show()

In [16]:
# group the data by platform
grouped = review_all.groupby('platform')

# create a Box trace for each platform
traces = []
for name, group in grouped:
    trace = go.Box(y=group['rating'], name=name)
    traces.append(trace)

# create the figure layout
layout = go.Layout(title='Side-by-Side Box Plot of Ratings by Platform', yaxis=dict(title='Rating'))

# create the figure object
fig = go.Figure(data=traces, layout=layout)

# show the figure
fig.show()

In [17]:
review_all = review_all.drop(['datecolumn'], axis=1)
review_all.head(5)

,platform,attraction,review,rating,coordinates,month,year
0,TripAdvisor,Alcatraz Island,My family took the tour ( BUY TICKETS IN ADVAN...,5.0,NaN,3.0,2023.0
1,TripAdvisor,Alcatraz Island,This is a must stop if you are in San Fran!!! ...,5.0,NaN,2.0,2023.0
2,TripAdvisor,Alcatraz Island,I did not expect to enjoy the tour as much as ...,5.0,NaN,3.0,2023.0
3,TripAdvisor,Alcatraz Island,San Francisco is completely unsafe. We bought ...,1.0,NaN,3.0,2023.0
4,TripAdvisor,Alcatraz Island,I had a 13-hour layover in San Francisco And I...,4.0,NaN,3.0,2023.0


## NLP on Reviews